In [3]:
%run dataFormating.ipynb

/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/emma/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
 

A Jupyter Widget

"00dbbdca-d86c-4bc9-803c-0602e0153f68"
"01e85778-2903-447b-bbab-dd750564ee2d"
"0916ecad-850e-4bfb-b8d4-e9e8f66c52d4"
"0ceae1e4-b8c8-4578-a157-4facbff9f899"
"15b73e25-5e0e-4216-943b-0279ae97b416"
"18102cfd-8ef0-42f1-9f05-30fc7779979c"
"1a03bc9e-bed4-4ddd-be7f-af23b1d5eb65"
"1f27519a-971f-4e39-bac7-9920bfc4b05b"
"24d954e3-23d4-4368-9b10-a1462d2c50d0"
"2734a37d-4ba5-454f-bf85-1f7b767138f6"
"2b91e6b2-d5ed-42e1-8bf5-dc23aeaf7e6a"
"3505d082-e62f-4d76-8dda-4ca6dc2d7c6c"
"3685a015-fa97-4457-ad73-da1c50210fe1"
"3d733347-0313-441a-b77c-3e4046042a53"
"3ef14300-4987-4b54-a56c-5b6d1f8a24a1"
"3fe0632f-b218-41c3-adfd-27083f271c19"
"44fe6cda-3f3e-4a6d-857e-4f22c3787202"
"4a859cce-5826-4807-9e1d-4dd5eba16180"
"4cd1abad-a077-46a1-bbe4-7a83594de09e"
"4dbc1a0a-e1d2-44dd-833c-62252a56bfd6"
"5151a69c-201b-45ec-954a-460a07443f96"
"58d22690-8604-41cf-a5b7-d71fb3b9ad5b"
"5977184a-1be2-4725-9b48-f2782dc03efb"
"5c4939b5-425b-4d19-b5d2-0384a515539e"
"5ca76986-9cac-4824-82ef-e20b81199e1a"
"5d9e5030-50dc-4fcd-be4b-

In [78]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

from ipywidgets import FloatProgress
from IPython.display import display

from math import *

from scipy import stats
from scipy.stats.mstats import normaltest

# Questionnaire only

## Can the answers to the scientific questions be used to predict if the questionnaire was filled before or after the game?

Note: I am using only decision tree methods here because other methods like naive bayes do not make sense on categorical data

### If scientific questions are coded by answers

In [71]:
# Select columns that correspond to scientific questions
scientificColumns = [x for x in list(defForms.columns.values) if x[0] == "Q"]

# Pick features and target
features = defForms.loc[:, scientificColumns]
target = defForms["temporality"].astype('int') 

In [72]:
# Classify using decision trees -accounts for the small size of the dataset and the categorical nature of the features
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.64376218323586754

In [73]:
# Classify using random forests -accounts for the small size of the dataset and the categorical nature of the features, limit overfitting
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.73830409356725146

In [74]:
# Classify using extra tree classifiers, more random than random forest methods
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.68031189083820676

Conclusion: Accuracy is around 70%. Not bad but we expected better (19/12/2017)

## If scientific questions are coded by correctedness

In [81]:
# Select columns that correspond to scientific questions
scientificColumns = [x for x in list(defCorrectedForms.columns.values) if x[0] == "Q"]

# Pick features and target
features = defCorrectedForms.loc[:, scientificColumns]
target = defCorrectedForms["temporality"].astype('int') 

In [82]:
# Classify using decision trees -accounts for the small size of the dataset and the categorical nature of the features
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.58089668615984402

In [83]:
# Classify using random forests -accounts for the small size of the dataset and the categorical nature of the features, limit overfitting
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.71832358674463936

In [84]:
# Classify using extra tree classifiers, more random than random forest methods
clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, features, target)
scores.mean()

0.62670565302144254

Conclusion: Accuracy is around 70%. Not bad but we expected better (19/12/2017)

# RedMetrics only

# Questionnaire and RedMetrics

In [75]:
# Assuming features independance
# Check if features roughly follow a Gaussian distribution
gaussianTest = stats.mstats.normaltest(features, axis = 0)
percentOfGaussianFeatures = gaussianTest[0].count() / len(gaussianTest[0])
percentOfGaussianFeatures

0.77952755905511806

In [87]:
# Lets try a Gaussian Naive Bayes classifier
clf = GaussianNB()
scores = cross_val_score(clf, features, target)
scores.mean()

0.73586744639376223